In [19]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('samsung_price.xlsx')

In [3]:
stock_list = pd.read_html('https://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13')[0]

In [4]:
df.head()

,날짜,종가,전일비,시가,고가,저가,거래량
0,2022.03.23,70500,200,70600,71200,70300,12332242
1,2022.03.22,70300,400,69900,70500,69900,9402666
2,2022.03.21,69900,800,70900,71000,69900,11169002
3,2022.03.18,70700,500,70600,70900,70200,14410038
4,2022.03.17,71200,800,71200,71800,70900,17646315


In [5]:
stock_list['종목코드'] = stock_list['종목코드'].map('{:06d}'.format)
stock_list.sort_values(by='종목코드', inplace=True)
stock_list.reset_index(drop=True, inplace=True)
stock_list.head()

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,동화약품,000020,의약품 제조업,"의약품 제조,판매,수출입",1976-03-24,12월,유준하,http://www.dong-wha.co.kr,서울특별시
1,KR모터스,000040,그외 기타 운송장비 제조업,"이륜차(오토바이) 제조,도매",1976-05-25,12월,노성석,http://www.krmotors.com,경상남도
2,경방,000050,종합 소매업,"섬유류(면사,면혼방사,면직물,면혼방직물,화섬사,화섬직물) 제조,도매,수출입",1956-03-03,12월,"김준, 김담",http://www.kyungbang.co.kr,서울특별시
3,메리츠화재,000060,보험업,"손해보험(화재,해상,운송보험)",1956-07-02,12월,김용범,http://www.meritzfire.com,서울특별시
4,삼양홀딩스,000070,기타 금융업,"지주회사,경영자문컨설팅",1968-12-27,12월,"엄태웅, 윤재엽",http://www.samyang.com,서울특별시


In [6]:
code_list = ['373220']

In [7]:
from bs4 import BeautifulSoup
import requests

In [20]:
def get_df(code_list):
    df = pd.DataFrame()
    for i in range(len(code_list)):
        code = code_list[i]
        url = f'https://finance.naver.com/item/sise_day.naver?code={code}'
        req = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
        html = BeautifulSoup(req.text, "lxml")
        pgrr = html.find('td', class_='pgRR')
        s = pgrr.a['href'].split('=')
        last_page = s[-1]
        for page in range(1, int(last_page)+1):
            req = requests.get(f'{url}&page={page}', headers={'User-agent': 'Mozilla/5.0'})
            df = pd.concat([df, pd.read_html(req.text, encoding = 'euc-kr')[0]], ignore_index=True)
            df.dropna(inplace=True)
            df.reset_index(drop=True, inplace=True)
                 
        # 전일대비 종가 등락률 계산
        close = df['종가']
        daily_rate = (close / close.shift(-1)-1)* 100
        df['일간변동률(%)'] = daily_rate
        
        # 전일대비 거래량 등락률 계산
        volume = df['거래량']
        volume_rate = (volume / volume.shift(-1)-1) * 100
        df['거래량증감률(%)'] = volume_rate
        
        # RSI 계산을 위한 전일비 등락값
        # 기존 전일비 값은 +, -의 구분이 없음
        diff = close - close.shift(-1)
        df['전일비등락'] = diff
        
        # RSI (14일 기준) 계산
        u = np.where(df['전일비등락'].diff(1) > 0, df['전일비등락'].diff(1), 0)
        d = np.where(df['전일비등락'].diff(1) < 0, df['전일비등락'].diff(1) *(-1), 0)
        au = df['전일비등락'].rolling(window=14, min_periods=14).mean()
        ad = df['전일비등락'].rolling(window=14, min_periods=14).mean()
        rsi = au.div(ad+au) * 100
        df['RSI'] = rsi
        
        """
        # RSI(14일 기준) 계산
        df['RSI'] = None
        for i in range(len(df)-14):
            au, au_cnt = 0,0
            ad, ad_cnt = 0,0
            for j in range(i, i+13):
                if df['전일비등락'][j] > 0:
                    au += df['전일비등락'][j]
                    au_cnt += 1
                elif df['전일비등락'][j] < 0:
                    ad += df['전일비등락'][j]
                    ad_cnt += 1
            au /= au_cnt
            ad /= ad_cnt
            rs = au / ad
            rsi = rs / (1+rs) * 100
            df['RSI'][i] = rsi
            """
        
    return df

In [21]:
test = get_df(code_list)

In [22]:
# 데이터가 없는 행 일괄 삭제
# test.dropna(inplace = True)
# 인덱스 재배열
# test.reset_index(drop=True, inplace=True)

In [23]:
test

,날짜,종가,전일비,시가,고가,저가,거래량,일간변동률(%),거래량증감률(%),전일비등락,RSI
0,2022.03.24,408000.0,1000.0,405000.0,410000.0,398000.0,316104.0,0.245700,-39.610040,1000.0,NaN
1,2022.03.23,407000.0,9500.0,403000.0,407500.0,398000.0,523438.0,2.389937,26.442851,9500.0,NaN
2,2022.03.22,397500.0,11500.0,388500.0,399000.0,388000.0,413972.0,2.979275,-36.821992,11500.0,NaN
3,2022.03.21,386000.0,4000.0,385000.0,397500.0,384000.0,655247.0,1.047120,1.759860,4000.0,NaN
4,2022.03.18,382000.0,6000.0,377500.0,384000.0,370500.0,643915.0,1.595745,-21.100258,6000.0,NaN
5,2022.03.17,376000.0,12500.0,374000.0,381500.0,373000.0,816118.0,3.438790,30.510388,12500.0,NaN
6,2022.03.16,363500.0,4000.0,367000.0,367500.0,360000.0,625328.0,1.112656,-41.256508,4000.0,NaN
7,2022.03.15,359500.0,4000.0,363000.0,370000.0,355000.0,1064506.0,-1.100413,-46.760831,-4000.0,NaN
8,2022.03.14,363500.0,27500.0,385000.0,385500.0,361500.0,1999479.0,-7.033248,10.384719,-27500.0,NaN
9,2022.03.11,391000.0,26500.0,411500.0,412000.0,390500.0,1811373.0,-6.347305,68.273467,-26500.0,NaN


In [12]:
len(test)

36

In [34]:
delta = test['전일비등락']

In [35]:
up = delta.copy()
up[up<0] = 0

In [36]:
down = delta.copy()
down[down>0] = 0

In [37]:
test['up'] = up

In [38]:
test['down'] = down

In [40]:
avg_gain = test['up'].rolling(window=14).mean()
avg_loss = abs(test['down'].rolling(window=14).mean())
rs = avg_gain / avg_loss
rsi = 100.0 - (100.0 / (1.0+rs))
test['RSI'] = rsi

In [41]:
test

,날짜,종가,전일비,시가,고가,저가,거래량,일간변동률(%),거래량증감률(%),전일비등락,RSI,up,down
0,2022.03.24,408000.0,1000.0,405000.0,410000.0,398000.0,316104.0,0.245700,-39.610040,1000.0,NaN,1000.0,0.0
1,2022.03.23,407000.0,9500.0,403000.0,407500.0,398000.0,523438.0,2.389937,26.442851,9500.0,NaN,9500.0,0.0
2,2022.03.22,397500.0,11500.0,388500.0,399000.0,388000.0,413972.0,2.979275,-36.821992,11500.0,NaN,11500.0,0.0
3,2022.03.21,386000.0,4000.0,385000.0,397500.0,384000.0,655247.0,1.047120,1.759860,4000.0,NaN,4000.0,0.0
4,2022.03.18,382000.0,6000.0,377500.0,384000.0,370500.0,643915.0,1.595745,-21.100258,6000.0,NaN,6000.0,0.0
5,2022.03.17,376000.0,12500.0,374000.0,381500.0,373000.0,816118.0,3.438790,30.510388,12500.0,NaN,12500.0,0.0
6,2022.03.16,363500.0,4000.0,367000.0,367500.0,360000.0,625328.0,1.112656,-41.256508,4000.0,NaN,4000.0,0.0
7,2022.03.15,359500.0,4000.0,363000.0,370000.0,355000.0,1064506.0,-1.100413,-46.760831,-4000.0,NaN,0.0,-4000.0
8,2022.03.14,363500.0,27500.0,385000.0,385500.0,361500.0,1999479.0,-7.033248,10.384719,-27500.0,NaN,0.0,-27500.0
9,2022.03.11,391000.0,26500.0,411500.0,412000.0,390500.0,1811373.0,-6.347305,68.273467,-26500.0,NaN,0.0,-26500.0
